In [ ]:
import torch
from transformers import BertModel
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# custom distloss
class DistLoss(nn.Module):
    def __init__(self):
        super(DistLoss, self).__init__()

    def forward(self, vector1, vector2):
        cos_sim = F.cosine_similarity(vector1, vector2, dim=1)
        divided = cos_sim / (torch.norm(vector1, dim=1) + 1)
        loss = torch.mean(divided)
        return loss

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')

loss_fn_first_layers = DistLoss()
loss_fn_last_layers = nn.CrossEntropyLoss()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# accuracy metric for our classififcation
# скорее рыба
def accuracy(model_labels, labels):
  return torch.mean((model_labels == labels).float())

In [ ]:
# Training loop
# пока считаю, что уже есть вектора маскированных предложений
def train(train_loader, model = BertModel.from_pretrained('bert-base-uncased'), num_epochs = 5, lr=2e-5, loss_fn_first_layers = loss_fn_first_layers, loss_fn_last_layers = loss_fn_last_layers, device = device):
  model = model
  train_loader = train_loader
  num_epochs = num_epochs
  optimizer = optim.Adam(model.parameters(), lr=lr)
  for epoch in range(num_epochs):
      model.train()
      total_loss = 0

      for batch in train_loader:
          input_neu, input_erg, target_neu, target_erg = batch

          # TO DO: transfer to device, get outputs

          optimizer.zero_grad()

          outputs = "sth"
          # Extract outputs from the first and last layers
          first_layers_output = model(inputs, output_hidden_states=True).hidden_states[:3]  # first three layers
          last_layers_output = model(inputs, output_hidden_states=True).hidden_states[3:]  # the rest of the layers

          # Calculate the loss for each group
          loss_first_layers = loss_fn_first_layers(first_layers_output, target)
          loss_last_layers = loss_fn_last_layers(last_layers_output, target)

          # Total loss
          # коэффициенты, нормировать arctg? динамично менять коэффициент,
          total_loss = loss_first_layers + loss_last_layers
          accuracy = accuracy(outputs, target)

          total_loss.backward()
          optimizer.step()

          total_loss += total_loss.item()

      print(f"Epoch {epoch + 1}, /n/n Total loss: {total_loss / len(train_loader)}, /n/n accuracy: {accuracy}")